In [1]:

# 필요 라이브러리

# 분석을 위한 라이브러리
import numpy as np
import pandas as pd

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import os

# 에러메세지 안나오게
import warnings
warnings.filterwarnings('ignore')

# pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',100)

In [2]:
import matplotlib

matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

In [4]:
log = pd.read_csv('logs.txt',index_col=0)
log

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen
0,OPEN,NoView,PDF,9400fd2e43d7dc2d054ca78806236ee1,LOCALSTORAGE,2016.7.18,Per_Dir
1,CLOSE,NoView,PDF,9400fd2e43d7dc2d054ca78806236ee1,LOCALSTORAGE,2016.7.18,Per_Dir
2,OPEN,View,PDF,9400fd2e43d7dc2d054ca78806236ee1,MYPOLARISDRIVE,2016.7.18,Pub_Dir
3,CLOSE,View,PDF,9400fd2e43d7dc2d054ca78806236ee1,MYPOLARISDRIVE,2016.7.18,Pub_Dir
4,OPEN,NoView,PDF,f191063c562691041dfa935ff0876975,OTHERAPP,2016.7.6,Main
...,...,...,...,...,...,...,...
301856,CLOSE,View,XLSX,2ed068d5e6a72e80e4a997c01c59f782,OTHERAPP,2016.7.7,Main
301857,OPEN,NoView,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5,Main
301858,RESET,View,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5,Main
301859,OPEN,NoView,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016.7.14,Main


In [5]:
# 데이터 정보 - 현재 결측치 없음.
log.info()

<class 'pandas.core.frame.DataFrame'>
Index: 301861 entries, 0 to 301860
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   actiontype        301861 non-null  object
 1   ismydoc           301861 non-null  object
 2   ext               301861 non-null  object
 3   sessionid         301861 non-null  object
 4   documentposition  301861 non-null  object
 5   datetime          301861 non-null  object
 6   screen            301861 non-null  object
dtypes: object(7)
memory usage: 18.4+ MB


In [10]:
# 컬럼들의 유니크값
for i in log.columns:
    print(f'{i}의 유니크값 : ',log[i].nunique())
    print(log[i].unique())
    print('-----------------------------------------------------')

actiontype의 유니크값 :  8
['OPEN' 'CLOSE' 'RESET' 'SAVEAS' 'SAVE' 'SAVEAS_OTHER' 'EXPORT_SAME'
 'EXPORT']
-----------------------------------------------------
ismydoc의 유니크값 :  2
['NoView' 'View']
-----------------------------------------------------
ext의 유니크값 :  16
['PDF' 'HWP' 'XLSX' 'PPT' 'DOCX' 'ODT' 'DOC' 'XLS' 'PPTX' 'TXT' 'PPS'
 'PPSX' 'WORD' 'PNG' 'JPG' 'SHEET']
-----------------------------------------------------
sessionid의 유니크값 :  114994
['9400fd2e43d7dc2d054ca78806236ee1' 'f191063c562691041dfa935ff0876975'
 '3747719d8f6bdd334703320433928d5a' ... '2ed068d5e6a72e80e4a997c01c59f782'
 '41b8c1df291edd2aec30fc610e5b676e' '9b5ee236571c2fcbe25132b6ba71d764']
-----------------------------------------------------
documentposition의 유니크값 :  7
['LOCALSTORAGE' 'MYPOLARISDRIVE' 'OTHERAPP' 'NEWDOCUMENT' 'NONE'
 'OTHERCLOUD' 'SHAREDDOCUMENT']
-----------------------------------------------------
datetime의 유니크값 :  30
['2016.7.18' '2016.7.6' '2016.7.20' '2016.7.28' '2016.7.19' '2016.7.24'
 '2016.

In [13]:
# session_id는 정수값으로 대체.
mapping = {}

# 주어진 문자열을 순회하면서 번호 매김
for item in log['sessionid']:
    if item in mapping:  # 이미 매핑된 문자인 경우
        pass
    else:  # 처음 나타나는 문자인 경우 새 번호를 부여
        new_number = len(mapping) + 1
        mapping[item] = new_number
    

log['sessionid'] = log['sessionid'].apply(lambda x : mapping[x])

In [11]:
# datetime은 데이터타입 변환
log.datetime = pd.to_datetime(log.datetime)

In [14]:
log

,actiontype,ismydoc,ext,sessionid,documentposition,datetime,screen
0,OPEN,NoView,PDF,1,LOCALSTORAGE,2016-07-18,Per_Dir
1,CLOSE,NoView,PDF,1,LOCALSTORAGE,2016-07-18,Per_Dir
2,OPEN,View,PDF,1,MYPOLARISDRIVE,2016-07-18,Pub_Dir
3,CLOSE,View,PDF,1,MYPOLARISDRIVE,2016-07-18,Pub_Dir
4,OPEN,NoView,PDF,2,OTHERAPP,2016-07-06,Main
...,...,...,...,...,...,...,...
301856,CLOSE,View,XLSX,114992,OTHERAPP,2016-07-07,Main
301857,OPEN,NoView,DOCX,114993,OTHERAPP,2016-07-05,Main
301858,RESET,View,DOCX,114993,OTHERAPP,2016-07-05,Main
301859,OPEN,NoView,PPT,114994,OTHERAPP,2016-07-14,Main


In [ ]:
# 날짜별 로그수
log.groupby